# Pandas pipelines

[Method chaining](https://tomaugspurger.github.io/method-chaining) is a great way for writing pandas code as it allows us to go from:

```python
raw_data = pd.read_parquet(...)
data_with_types = set_dtypes(raw_data)
data_without_outliers = remove_outliers(data_with_types)
```

to 

```python
data = (
    pd.read_parquet(...)
    .pipe(set_dtypes)
    .pipe(remove_outliers)
)
```

But it does come at a cost, mostly in our ability to debug long pipelines. If there's a mistake somewhere along the way, you can only inspect the end result and lose the ability to inspect intermediate results. A mitigation for this is to add decorators to your pipeline functions that log common attributes of your dataframe on each step:


## Logging in method chaining
In order to use the logging capabilitites we first need to ensure we have a proper logger configured. We do this by running `logging.basicConfig(level=logging.DEBUG)`. 



In [ ]:
from sklego.datasets import load_chicken
from sklego.pandas_utils import log_step
chickweight = load_chicken(give_pandas=True)

In [ ]:
import logging

logging.basicConfig(level=logging.DEBUG)

If we now add a `log_step` decorator to our pipeline function and execute the function, we see that we get some logging statements for free

In [ ]:
@log_step
def set_dtypes(chickweight):
    return chickweight.assign(
        diet=lambda d: d['diet'].astype('category'),
        chick=lambda d: d['chick'].astype('category'),
    )

In [ ]:
chickweight.pipe(set_dtypes).head()

We can choose to log at different log levels. For example if we have a `remove_outliers` function that calls different outlier removal functions for different types of outliers, we might in general be only interested in the total outliers removed. In order to get that, we set the log level for our specific implementations to `logging.DEBUG`

In [ ]:
@log_step(level=logging.DEBUG)
def remove_dead_chickens(chickweight):
    dead_chickens = chickweight.groupby('chick').size().loc[lambda s: s < 12]
    return chickweight.loc[lambda d: ~d['chick'].isin(dead_chickens)]


@log_step
def remove_outliers(chickweight):
    return chickweight.pipe(remove_dead_chickens)

In [ ]:
chickweight.pipe(set_dtypes).pipe(remove_outliers).head()

We can now easily switch between log levels to get the full detail or the general overview

In [ ]:
logging.getLogger(__name__).setLevel(logging.INFO)
chickweight.pipe(set_dtypes).pipe(remove_outliers).head()